In [15]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [16]:
#test us super big?
#change up the order to have more data available for training
#this is not a bug/typo
test_data, validation_data, train_data = tfds.load('oxford_flowers102', split=['train', 'validation', 'test'], as_supervised=True)

TARGET_SIZE = (128, 128)
TARGET_SHAPE = (TARGET_SIZE[0], TARGET_SIZE[1], 3)

train_data = train_data.map(lambda x, y: (tf.image.resize(x, TARGET_SIZE), y))
validation_data = validation_data.map(lambda x, y: (tf.image.resize(x, TARGET_SIZE), y))
test_data = test_data.map(lambda x, y: (tf.image.resize(x, TARGET_SIZE), y))


steps = 41
train_data = train_data.shuffle(1024) #.batch(150)
validation_data = validation_data.shuffle(1024) #.batch(25)



print("Number of training samples: %d" % tf.data.experimental.cardinality(train_data))
print("Number of validation samples: %d" % tf.data.experimental.cardinality(validation_data))
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_data))

Number of training samples: 6149
Number of validation samples: 1020
Number of test samples: 1020


In [18]:
input = keras.Input(shape=TARGET_SHAPE)
normalizer = keras.layers.experimental.preprocessing.Normalization()
preprocess = normalizer(input)
normalizer.set_weights([np.array([0] * 3), np.array([255**2] * 3)])

#image_batch, label_batch = next(iter(train_data))
#test = normalizer(image_batch)
#plt.imshow(tf.reshape(test, (128, 128, 3)))

In [19]:
base_model = keras.applications.ResNet101V2(include_top=False, weights='imagenet', pooling='avg', input_shape=TARGET_SHAPE)
base_model.trainable = False
#print(base_model.summary())
#img = train_data.take(1)
#print(img)
#image_batch, label_batch = next(iter(train_data))
#print(image_batch)
#plt.imshow(image_batch / 255.0)
#feature_batch = base_model(image_batch)
#print(feature_batch.shape)

features = base_model(preprocess, training=False)
classifier_head = keras.layers.Dense(102)

outputs = classifier_head(features)

model = keras.Model(input, outputs)

171319296/171317808 [==============================] - 4s 0us/step


In [20]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

#print(model.summary())
#I cannot fathom why but this only works when the validation batch size is 150????
model.fit(train_data.batch(150)
, epochs=10
, validation_data=validation_data.batch(150)
, validation_steps=7
, steps_per_epoch=41
)

Epoch 1/10
41/41 [==============================] - 16s 382ms/step - loss: 3.1054 - accuracy: 0.3188 - val_loss: 1.9187 - val_accuracy: 0.4951
Epoch 2/10
41/41 [==============================] - 13s 317ms/step - loss: 1.2188 - accuracy: 0.7055 - val_loss: 1.4001 - val_accuracy: 0.6304
Epoch 3/10
41/41 [==============================] - 13s 320ms/step - loss: 0.8215 - accuracy: 0.8107 - val_loss: 1.2079 - val_accuracy: 0.6824
Epoch 4/10
41/41 [==============================] - 12s 304ms/step - loss: 0.6185 - accuracy: 0.8696 - val_loss: 1.1155 - val_accuracy: 0.7216
Epoch 5/10
41/41 [==============================] - 13s 315ms/step - loss: 0.4906 - accuracy: 0.9024 - val_loss: 1.0570 - val_accuracy: 0.7245
Epoch 6/10
41/41 [==============================] - 13s 313ms/step - loss: 0.4015 - accuracy: 0.9263 - val_loss: 1.0116 - val_accuracy: 0.7422
Epoch 7/10
41/41 [==============================] - 13s 317ms/step - loss: 0.3316 - accuracy: 0.9483 - val_loss: 0.9818 - val_accuracy: 0.7490